<a href="https://colab.research.google.com/github/zeinabkamkar98/graph_simulation/blob/main/graph_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step Zero

- Loading libs
- datasets
- basic utils

**Loading Libraries**

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import  rel_entr

import networkx as nx
from networkx.drawing import draw_networkx

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


**Load github datasets**

If you wanna use github datasets run below cell. This datasets are limited (10 datasets), They're just for test.

In [2]:
graphs_name = ['DHFR','BZR','COX2','AIDS','ENZYMES','DD','MUTAG','NCI1','PROTEINS_full','PTC_MR']

!git clone https://github.com/zeinabkamkar98/graph_simulation.git

fatal: destination path 'graph_simulation' already exists and is not an empty directory.


 **Utils**


In [3]:
def read_graph_file(path):
    G = nx.Graph()
    data = np.loadtxt(path, delimiter=',').astype(int)
    data_tuple = list(map(tuple, data))
    G.add_edges_from(data_tuple)
    return G


def create_random_graph(nodes_count, edges_count ):
    return nx.gnm_random_graph(nodes_count, edges_count)


def add_features_to_graph(G: nx.Graph):

    for node in G.nodes:
        neighbors = G.neighbors(node)
        degrees = [G.degree(neighbor) for neighbor in neighbors]
        min_degree = np.min(degrees) if degrees else 0
        max_degree = np.max(degrees) if degrees else 0
        mean_degree = np.mean(degrees) if degrees else 0
        median_degree = np.median(degrees) if degrees else 0
        G.nodes[node]["features"] = np.array(
            [
                mean_degree,
                median_degree,
                max_degree,
                min_degree,
            ]
        )
    for edge in G.edges:
        G.edges[edge]["features"] = np.zeros(5)
        G.edges[edge]["features"][:2] = (
            G.nodes[edge[0]]["features"][:2] + G.nodes[edge[1]]["features"][:2]
        )

        max_deg = max(G.nodes[edge[0]]["features"][2], G.nodes[edge[1]]["features"][2])
        min_deg = min(G.nodes[edge[0]]["features"][3], G.nodes[edge[1]]["features"][3])
        G.edges[edge]["features"][2] = max_deg
        G.edges[edge]["features"][3] = min_deg
        G.edges[edge]["features"][4] = max_deg - min_deg

def get_graph_features(graph):
    return np.concatenate(
            [np.array([v]) for v in nx.get_edge_attributes(graph, "features").values()]
        )
def calculate_divergence(g1, g2):
    epsilon = 1e-10

    g1_features_data = get_graph_features(g1)
    g2_features_data = get_graph_features(g2)

    feature_divergence = np.zeros(g1_features_data.shape[1])

    for i in range(5):
      g1_hist, _ = np.histogram(g1_features_data[:, i])
      g2_hist, _ = np.histogram(g2_features_data[:, i])
      feature_divergence[i] = np.sum(
          rel_entr(g1_hist + epsilon / (np.sum(g1_hist) + epsilon) , g2_hist + epsilon / (np.sum(g2_hist) + epsilon) )
      )

    return feature_divergence

def convet_to_data_frame(data):
  column_labels = ["mean", "median", "max", "min", "range"]

  return pd.DataFrame(data, columns = column_labels)



# Step One

Calculate Features for Original Graph and its complement


**Original graph**

Import original graph and calculate and features to its nodes and edges

In [4]:
selected_graph_name= graphs_name[1]
graph = read_graph_file('graph_simulation/DATASETS/MUTAG/MUTAG_A.txt')

add_features_to_graph(graph)

**Complement Of Original Graph**


In [5]:
graph_complement = nx.complement(graph)
add_features_to_graph(graph_complement)


# Step Two

generate a random graph with same node and edge number of the original graph and calculate divergence

In [6]:
random_seed = random.randint(1, 1000)
random.seed(random_seed)

simulated_graph = create_random_graph(len(graph.nodes), len(graph.edges))
add_features_to_graph(simulated_graph)
simulated_graph_div = np.sum(calculate_divergence(graph, simulated_graph))

print("simulated_graph_div", simulated_graph_div)

simulated_graph_comp = nx.complement(simulated_graph)
add_features_to_graph(simulated_graph_comp)
simulated_graph_comp_div = np.sum(calculate_divergence(graph_complement, simulated_graph_comp))

print("simulated_graph_comp", simulated_graph_comp_div)


simulated_graph_div 17846.207437399
simulated_graph_comp 10974549.883801207


# Step Three

- convert graphs to dataframes


Convert original graph to dataframe

In [7]:
graph_feature_data_frame = convet_to_data_frame(get_graph_features(graph))
graph_feature_data_frame['label'] = 1
graph_feature_data_frame.head()


,mean,median,max,min,range,label
0,4.000000,4.0,2.0,2.0,0.0,1
1,4.500000,4.5,3.0,2.0,1.0,1
2,4.500000,4.5,3.0,2.0,1.0,1
3,5.166667,5.5,3.0,2.0,1.0,1
4,5.000000,5.0,3.0,2.0,1.0,1


Convert original graph complement to data frame


In [8]:
graph_compelement_feature_data_frame = convet_to_data_frame(get_graph_features(graph_complement))
graph_compelement_feature_data_frame['label'] = 0
graph_compelement_feature_data_frame.tail()


,mean,median,max,min,range,label
5676409,6735.585214,6736.0,3369.0,3366.0,3.0,0
5676410,6735.584558,6736.0,3369.0,3366.0,3.0,0
5676411,6735.584682,6736.0,3369.0,3366.0,3.0,0
5676412,6735.584682,6736.0,3369.0,3366.0,3.0,0
5676413,6735.584446,6736.0,3369.0,3366.0,3.0,0


Mix both data farme


In [9]:
train_data_frame = pd.concat([graph_feature_data_frame,graph_compelement_feature_data_frame])
train_data_frame = train_data_frame.reset_index(drop=True)
train_data_frame

,mean,median,max,min,range,label
0,4.000000,4.0,2.0,2.0,0.0,1
1,4.500000,4.5,3.0,2.0,1.0,1
2,4.500000,4.5,3.0,2.0,1.0,1
3,5.166667,5.5,3.0,2.0,1.0,1
4,5.000000,5.0,3.0,2.0,1.0,1
...,...,...,...,...,...,...
5680130,6735.585214,6736.0,3369.0,3366.0,3.0,0
5680131,6735.584558,6736.0,3369.0,3366.0,3.0,0
5680132,6735.584682,6736.0,3369.0,3366.0,3.0,0
5680133,6735.584682,6736.0,3369.0,3366.0,3.0,0


Convert simulated graph to data frame

In [10]:
simulated_graph_feature_data_frame = convet_to_data_frame(get_graph_features(simulated_graph))

simulated_graph_feature_data_frame.head()

,mean,median,max,min,range
0,5.000000,5.0,3.0,2.0,1.0
1,5.500000,5.5,4.0,2.0,2.0
2,3.000000,3.0,2.0,1.0,1.0
3,7.166667,6.5,6.0,2.0,4.0
4,6.000000,6.0,4.0,2.0,2.0


Convert complement of the simulated graph to data frame



In [11]:
simulated_graph_comp_feature_data_frame = convet_to_data_frame(get_graph_features(simulated_graph_comp))
simulated_graph_comp_feature_data_frame.tail()

,mean,median,max,min,range
5676409,6735.587702,6736.0,3370.0,3361.0,9.0
5676410,6735.586575,6736.0,3370.0,3361.0,9.0
5676411,6735.587293,6736.0,3370.0,3361.0,9.0
5676412,6735.586166,6736.0,3370.0,3361.0,9.0
5676413,6735.588419,6736.0,3370.0,3361.0,9.0


Mix both simulated graph and simulated graph complement

In [12]:
simulate_data_frame = pd.concat([simulated_graph_feature_data_frame,simulated_graph_comp_feature_data_frame])
simulate_data_frame = simulate_data_frame.reset_index(drop=True)
simulate_data_frame

,mean,median,max,min,range
0,5.000000,5.0,3.0,2.0,1.0
1,5.500000,5.5,4.0,2.0,2.0
2,3.000000,3.0,2.0,1.0,1.0
3,7.166667,6.5,6.0,2.0,4.0
4,6.000000,6.0,4.0,2.0,2.0
...,...,...,...,...,...
5680130,6735.587702,6736.0,3370.0,3361.0,9.0
5680131,6735.586575,6736.0,3370.0,3361.0,9.0
5680132,6735.587293,6736.0,3370.0,3361.0,9.0
5680133,6735.586166,6736.0,3370.0,3361.0,9.0


# Step Four

- train logistic regression model
- predict simulated graph edge labels with the trained model

Train Logistic Tegression model with the orginal graph edges features


In [13]:
X = train_data_frame.drop('label', axis=1)
y = train_data_frame['label']

# split the train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=23)
y_test.sum()

# LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

# Prediction
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Logistic Regression model accuracy (in %):", acc*100)


Logistic Regression model accuracy (in %): 99.94824071237682


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predict label of the simulated graph's edges label

In [14]:
y_pred = clf.predict(simulate_data_frame)
predicted_graph = simulate_data_frame
predicted_graph['label'] = y_pred
predicted_graph

,mean,median,max,min,range,label
0,5.000000,5.0,3.0,2.0,1.0,0
1,5.500000,5.5,4.0,2.0,2.0,0
2,3.000000,3.0,2.0,1.0,1.0,0
3,7.166667,6.5,6.0,2.0,4.0,1
4,6.000000,6.0,4.0,2.0,2.0,0
...,...,...,...,...,...,...
5680130,6735.587702,6736.0,3370.0,3361.0,9.0,0
5680131,6735.586575,6736.0,3370.0,3361.0,9.0,0
5680132,6735.587293,6736.0,3370.0,3361.0,9.0,0
5680133,6735.586166,6736.0,3370.0,3361.0,9.0,0


# Step Five

Make new simulated graph based on the predicted labels

Find edges nodes numbers

In [15]:
edges_nodes_data_frame = pd.DataFrame(list(simulated_graph.edges) + list(simulated_graph_comp.edges), columns=['source', 'target'])
edges_nodes_data_frame

,source,target
0,0,1993
1,0,2737
2,1,1803
3,2,743
4,2,431
...,...,...
5680130,3367,3369
5680131,3367,3370
5680132,3368,3369
5680133,3368,3370


Create new graph based on the result


In [16]:
predicted_graph_with_edge_info = combined_df = pd.concat([predicted_graph, edges_nodes_data_frame], axis=1)
predicted_graph_with_edge_info

,mean,median,max,min,range,label,source,target
0,5.000000,5.0,3.0,2.0,1.0,0,0,1993
1,5.500000,5.5,4.0,2.0,2.0,0,0,2737
2,3.000000,3.0,2.0,1.0,1.0,0,1,1803
3,7.166667,6.5,6.0,2.0,4.0,1,2,743
4,6.000000,6.0,4.0,2.0,2.0,0,2,431
...,...,...,...,...,...,...,...,...
5680130,6735.587702,6736.0,3370.0,3361.0,9.0,0,3367,3369
5680131,6735.586575,6736.0,3370.0,3361.0,9.0,0,3367,3370
5680132,6735.587293,6736.0,3370.0,3361.0,9.0,0,3368,3369
5680133,6735.586166,6736.0,3370.0,3361.0,9.0,0,3368,3370


Remove edges with label zero

In [17]:
new_graph_edges_data_frame = predicted_graph_with_edge_info[predicted_graph_with_edge_info['label'] != 0]
new_graph_edges_data_frame

,mean,median,max,min,range,label,source,target
3,7.166667,6.5,6.0,2.0,4.0,1,2,743
5,8.714286,8.0,7.0,1.0,6.0,1,3,1325
6,9.400000,9.0,7.0,2.0,5.0,1,3,20
8,5.866667,6.0,5.0,1.0,4.0,1,4,2086
9,7.666667,7.5,6.0,2.0,4.0,1,4,2747
...,...,...,...,...,...,...,...,...
3707,5.333333,6.0,4.0,1.0,3.0,1,3131,3256
3708,6.416667,5.5,6.0,2.0,4.0,1,3135,3168
3712,6.250000,6.0,5.0,1.0,4.0,1,3159,3367
3713,6.000000,5.0,5.0,1.0,4.0,1,3165,3316


Create new simulated graph based on the result in previous step and calculate divergence

In [18]:
new_simulated_graph = nx.from_pandas_edgelist(new_graph_edges_data_frame, source='source', target='target')

add_features_to_graph(new_simulated_graph)
new_simulated_graph_div = np.sum(calculate_divergence(graph, new_simulated_graph))

print("new_simulated_graph_div", new_simulated_graph_div)

new_simulated_graph_comp = nx.complement(new_simulated_graph)
add_features_to_graph(new_simulated_graph_comp)
new_simulated_graph_comp_div = np.sum(calculate_divergence(graph_complement, new_simulated_graph_comp))

print("new_simulated_graph_comp", simulated_graph_comp_div)


new_simulated_graph_div 33665.29815774934
new_simulated_graph_comp 10974549.883801207
